# 4. Advanced GPT-4 And ChatGPT Techniques

In [2]:
import settings
import openai
import pandas as pd

## Fine-Tuning with the OpenAI API

### Uploading a new file

In [13]:
ft_file = openai.files.create(
    file=open("files/out_openai_completion_prepared.jsonl", "rb"),
    purpose="fine-tune"
)

In [ ]:
openai.fine_tunes.create(
    training_file=ft_file.id,
    model="davinci",
    suffix="direct_marketing"
)

In [19]:
fts = openai.fine_tunes.list()
print(fts)

SyncPage[FineTune](data=[FineTune(id='ft-rELqGCDP2fLeRKOzjrMZoWcX', created_at=1704595520, fine_tuned_model='davinci:ft-personal:direct-marketing-2024-01-07-02-51-27', hyperparams=Hyperparams(batch_size=1, learning_rate_multiplier=0.1, n_epochs=4, prompt_loss_weight=0.01, classification_n_classes=None, classification_positive_class=None, compute_classification_metrics=None), model='davinci', object='fine-tune', organization_id='org-kzSPykuhYsOfIvA9MVG4LOU8', result_files=[FileObject(id='file-misMpvuJ5Bg606ZFP2eCg0kD', bytes=35537, created_at=1704595889, filename='compiled_results.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)], status='succeeded', training_files=[FileObject(id='file-B0RRlOAlj4xLrqP0YHvLBpNC', bytes=77420, created_at=1704595518, filename='out_openai_completion_prepared.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)], updated_at=1704595889, validation_files=[], events=None), FineTune(id='ft

### Creating a fine-tuned model

In [9]:
def chat_completion(prompt, model="gpt-4", temperature=0):
    res = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    return res.choices[0].message.content
    

In [ ]:
l_sector = ['Grocery Stores', 'Restaurants', 'Fast Food Restaurants', 'Pharmacies', 'Service Stations (Fuel)', 'Electronics Stores']
l_city = ['Brussels', 'Paris', 'Berlin']
l_size = ['small', 'medium', 'large']


f_prompt = """ 
Role: You are an expert content writer with extensive direct marketing 
experience. You have strong writing skills, creativity, adaptability to 
different tones and styles, and a deep understanding of audience needs and
preferences for effective direct campaigns.
Context: You have to write a short message in no more than 2 sentences for a
direct marketing campaign to sell a new e-commerce payment service to stores. 
The target stores have the following three characteristics:
- The sector of activity: {sector}
- The city where the stores are located: {city} 
- The size of the stores: {size}
Task: Write a short message for the direct marketing campaign. Use the skills
defined in your role to write this message! It is important that the message
you create takes into account the product you are selling and the
characteristics of the store you are writing to.
"""

f_sub_prompt = "{sector}, {city}, {size}"


df = pd.DataFrame()
for sector in l_sector:
    for city in l_city:
        for size in l_size:
            for i in range(3): 
                prompt = f_prompt.format(sector=sector, city=city, size=size)
                sub_prompt = f_sub_prompt.format(sector=sector, city=city, size=size)
                response_txt = chat_completion(
                    prompt, model="gpt-3.5-turbo", temperature=1
                )
                new_row = {"prompt": sub_prompt, "completion": response_txt}
                new_row = pd.DataFrame([new_row])
                df = pd.concat([df, new_row], axis=0, ignore_index=True)

df.to_csv("files/out_openai_completion.csv",  index=False)

### Using the fine-tuned model

In [ ]:
openai.completions.create(
    model="",
    prompt="Hotel, New York, small ->",
    max_tokens=100,
    temperature=0,
    stop="\n"
)